In [1]:
#操作目录和文件
import os; nb_dir = os.getcwd(); 
#导入shp
import shapefile as shpf; 
#处理几何信息
import shapely; 
import shapely.geometry as shplgeo, shapely.affinity as shplaffin; 
import shapely.ops as shplops; 
#数值计算和函数处理
import numpy as np; 
from functools import partial; 
#投影变换
import pyproj; 
#用户界面
import tkinter as tk; 
import tkintermapview as tkmvw; 

In [2]:
#初始化窗口
wnd_main = tk.Tk(); 
wnd_main.title("HZAU Path Distribution"); 
wnd_main.geometry("800x600"); 
wnd_main.resizable(width=False, height=False); 

In [3]:
#添加顶端菜单栏
mnu_main_top = tk.Menu(wnd_main); 
wnd_main.config(menu = mnu_main_top); 
mnu_main_File = tk.Menu(wnd_main); 
mnu_main_top.add_cascade(label = "File", menu = mnu_main_File)

In [4]:
#添加地图界面
#禁用联网加载底图瓦片服务(将网址设置为"空白页")
#警告! OpenStreetMap中的部分制图要素, 不符合中国现行法律法规
map_main_geo = tkmvw.TkinterMapView(wnd_main, width=800, height=600); 
map_main_geo.set_tile_server("about:blank"); 
map_main_geo.set_position(30.475, 114.35); 
map_main_geo.min_zoom = 15; map_main_geo.max_zoom = 19; 

In [5]:
#导入华中农业大学2022年的道路网(已去除GCJ偏移)
HZAU_Path_Minor = shpf.Reader(os.sep.join([nb_dir, 
    "HZAU_Path_Minor", "Path_Minor_Adj_02_CGCS.shp"]))
HZAU_Path_Minor_shp = tuple(HZAU_Path_Minor.iterShapes());
HZAU_Path_Minor_shplgeo = []; 
HZAU_Path_Minor_shplenvl = []; 
for sgm in HZAU_Path_Minor_shp: 
    sgm_shplgeo = shplgeo.LineString(sgm.points); 
    HZAU_Path_Minor_shplgeo.append(sgm_shplgeo); 
    HZAU_Path_Minor_shplenvl.append(sgm_shplgeo.envelope); 

In [6]:
#获取地图界面当前显示区域的最小, 最大经纬度
def get_boundary_latlon(mapview: tkmvw.map_widget.TkinterMapView):
    ymin, xmin = mapview.convert_canvas_coords_to_decimal_coords(
        0, mapview.height); 
    ymax, xmax = mapview.convert_canvas_coords_to_decimal_coords(
        mapview.width, 0); 
    return(xmin, ymin, xmax, ymax); 

In [7]:
#更新绘制结果
def redraw_annotation(event, mapview: tkmvw.map_widget.TkinterMapView, 
    geoms, envls = None): 
    #擦除原来绘制的路网
    mapview.delete_all_path(); 
    #计算地图界面当前的显示范围
    xmin, ymin, xmax, ymax = get_boundary_latlon(mapview); 
    visible_region = shplgeo.box(xmin, ymin, xmax, ymax); 
    if envls == None: 
        envls = [sgm_shplgeo.envelope for sgm_shplgeo in geoms]; 
    tolerance = abs(xmax - xmin + (ymax - ymin) * 1j) / 500; 
    for sgm_shplgeo, envl in zip(geoms, envls): 
        #目标路段外接矩形与显示区域相交, 是路段部分位于显示区域内的必要不充分条件
        if envl.intersects(visible_region): 
            #使用地图当前的显示范围对路段进行裁剪
            geoms_clipped = shplops.clip_by_rect(
                sgm_shplgeo, xmin, ymin, xmax, ymax
            ); 
            #将路段裁剪后的每一部分按照折线上点的顺序绘制在地图界面上
            if isinstance(geoms_clipped, shplgeo.linestring.LineString): 
                geoms_clipped = shplgeo.MultiLineString([geoms_clipped]); 
            elif isinstance(geoms_clipped, shplgeo.point.Point): 
                geoms_clipped = shplgeo.MultiPoint([geoms_clipped]); 
            if isinstance(geoms_clipped, shplgeo.multilinestring.MultiLineString): 
                for part in geoms_clipped: 
                    x_list, y_list = part.simplify(tolerance).coords.xy; 
                    mapview.set_path(list(zip(tuple(y_list), tuple(x_list)))); 
            elif isinstance(geoms_clipped, shplgeo.multipoint.MultiPoint): 
                for part in geoms_clipped: 
                    x_list, y_list = part.xy; 
                    mapview.set_marker(y_list, x_list); 

In [8]:
#为用户交互事件指定要执行的命令
map_main_redraw = partial(redraw_annotation, 
    mapview=map_main_geo, geoms=HZAU_Path_Minor_shplgeo, 
    envls=HZAU_Path_Minor_shplenvl); 
#用户拖拽地图, 或者点击界面上的缩放按钮缩放地图
wnd_main.bind("<B1-ButtonRelease>", map_main_redraw); 
#用户从其他界面切换至当前界面
wnd_main.bind("<FocusIn>", map_main_redraw); 
#用户通过鼠标滚轮缩放地图
wnd_main.bind("<MouseWheel>", map_main_redraw); 

In [9]:
map_main_geo.pack();
wnd_main.mainloop(); 